In [23]:
import os
import pickle
import numpy as np
import tensorflow as tf
import keras as K
import jams
import librosa
import matplotlib.pyplot as plt

# (Optional) if you need to make tf accessible as in your original code:
import builtins
builtins.tf = tf




In [24]:
# Set your working directory (adjust to your environment)
working = '/Users/manuel/working/'
split = 0  # choose the appropriate split
epochs = 5  # choose the appropriate epochs the model was trained on

In [25]:
def load_model(model_spec_path, weights_path):
    # Load the serialized model spec
    with open(model_spec_path, "rb") as fd:
        model_spec = pickle.load(fd)
    # Reconstruct the model from the serialized spec
    model = K.utils.deserialize_keras_object(model_spec, safe_mode=False)
    # Load the weights from the saved file
    model.load_weights(weights_path)
    return model

# Set parameters for the model you wish to load (adjust split/epochs as needed)
output_path = os.path.join(working, 'chords/model_deep')


model_spec_path = os.path.join(output_path, "fold{:02d}_model_{:03d}_epochs.pkl".format(split, epochs))
weights_path = os.path.join(output_path, "fold{:02d}_weights_{:03d}_epochs.keras".format(split, epochs))

# Load and display model summary
model = load_model(model_spec_path, weights_path)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cqt_mag (InputLayer)            │ (None, None, 216, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 216, 1)   │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, None, 216, 1)   │            26 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, None, 1, 36)    │         7,812 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, None, 36)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 256)      │       127,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, None, 256)      │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ chord_tag (TimeDistributed)     │ (None, None, 170)      │        43,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,426,402 (5.44 MB)

 Trainable params: 475,466 (1.81 MB)

 Non-trainable params: 2 (8.00 B)

 Optimizer params: 950,934 (3.63 MB)

In [26]:
def rename_slashes_in_op_fields(op):
    """
    In-place rename of all slash-laden keys inside op.fields
    so that the new keys replace '/' with '_'.
    """
    if hasattr(op, "fields") and isinstance(op.fields, dict):
        new_dict = {}
        for old_key, old_val in op.fields.items():
            new_key = old_key.replace("/", "_")  # e.g. 'cqt/mag' -> 'cqt_mag'
            new_dict[new_key] = old_val
        op.fields = new_dict

def rename_slashes_in_pump_opmap(pump):
    for op_name, op in pump.opmap.items():
        rename_slashes_in_op_fields(op)

def rename_slashes_in_pump_ops_list(pump):
    for op in pump.ops:
        rename_slashes_in_op_fields(op)

# Load the pump from your saved file
pump_path = os.path.join(working, 'chords/pump.pkl')
with open(pump_path, "rb") as fd:
    pump = pickle.load(fd)

# Rename keys in the pump operators
rename_slashes_in_pump_opmap(pump)
rename_slashes_in_pump_ops_list(pump)


In [27]:
def compute_cqt_mag(audio_path, sr=44100, hop_length=4096, n_bins=216, bins_per_octave=36, use_log=True):
    # Load the audio
    y, _ = librosa.load(audio_path, sr=sr)
    # Compute the constant-Q transform (CQT)
    cqt = librosa.cqt(y, sr=sr, hop_length=hop_length, n_bins=n_bins, bins_per_octave=bins_per_octave)
    # Compute magnitude
    cqt_mag = np.abs(cqt)
    # Apply logarithmic compression if needed (as in your original settings)
    if use_log:
        cqt_mag = np.log1p(cqt_mag)
    # Transpose so that time frames come first and add channel dimension: (time, 216, 1)
    cqt_mag = np.expand_dims(cqt_mag.T, axis=-1)
    return cqt_mag

# Example usage: specify your new audio file path here
new_audio_path = os.path.join(working, 'test_new_sounds/C-real-piano.wav')


In [28]:
# Compute the CQT magnitude feature for the new audio
cqt_mag_feature = compute_cqt_mag(new_audio_path)

# Display feature shape (should be (n_frames, 216, 1))
print("Feature shape:", cqt_mag_feature.shape)

Feature shape: (150, 216, 1)


In [29]:
# Add a batch dimension: (1, n_frames, 216, 1)
input_data = np.expand_dims(cqt_mag_feature, axis=0)

# Predict using the loaded model (the output is per time frame)
predictions = model.predict(input_data)[0]

# Use the pump's chord_tag transformer to convert predictions to chord labels
chord_annotation = pump["chord_tag"].inverse(predictions)

print("Predicted chord annotation:")
print(chord_annotation)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted chord annotation:
{
  "annotation_metadata": {
    "curator": {
      "name": "",
      "email": ""
    },
    "annotator": {},
    "version": "",
    "corpus": "",
    "annotation_tools": "",
    "annotation_rules": "",
    "validation": "",
    "data_source": ""
  },
  "namespace": "chord",
  "data": [
    {
      "time": 0.0,
      "duration": 0.09287981859410431,
      "value": "C:aug",
      "confidence": 0.10223222523927689
    },
    {
      "time": 0.09287981859410431,
      "duration": 13.653333333333332,
      "value": "C:maj",
      "confidence": 0.18624819815158844
    },
    {
      "time": 13.746213151927437,
      "duration": 0.2786394557823133,
      "value": "C:aug",
      "confidence": 0.11003585159778595
    }
  ],
  "sandbox": {},
  "time": 0,
  "duration": null
}
